# **Study of the Transmission Rate $\color{blue}\beta$**

During our study, we developped some tools in order to analyze and understand the behaviour of the Covid-19 pandemic. \\
However, we still need to study its own specific rates like $\color{blue}\beta$ and $\color{green}\gamma$ to draw relevant conclusions about the response to governmental measures. \\
The transmission rate $\color{blue}{\beta}$ is a key parameter in the study of Covid-19. Since there are many methods to determine its value, the objective of this Notebook is to implement several of them, and then compare to find precise values.

# **1st Approach to get Beta: the Linearization**

A first and simple idea is to calculate $\beta$ with the given set of equations, replacing derivative values by linearized ones. 

## **Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## **Utilitary Functions**

In [ ]:
def graphics_plot(t, y, y_title, title):
  # Plotting the results

  fig = go.Figure()

  for i in range(len(y)):
    if i<4:
      fig.add_trace(go.Scatter(
      name=y_title[i],
      x=t,
      y=y[i],
      mode='lines',
      marker_symbol='circle',
      line=dict(width=3, dash="dash")
      ))
    else:
      fig.add_trace(go.Scatter(
      name=y_title[i],
      x=t,
      y=y[i],
      mode='lines',
      marker_symbol='circle',
      line=dict(width=3, dash="solid")
      ))
  
  fig.update_layout(
      template='xgridoff',
      xaxis=dict(showgrid=False),
      xaxis_title='Date',
      legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.01,
        xanchor='right',
        x=0.95
        ),
      title_text=title)

  fig.show()

## **Theory**

Regarding the equational system: 

$$ \begin{split}
   \frac{dS(t)}{dt} & = \frac{-\beta I(t) S(t)}{N}
   \end{split} $$

\begin{split}
\frac{dI(t)}{dt} & = \frac{\beta I(t) S(t)}{N} -\gamma I(t) -\mu I(t)
\end{split}

\begin{split}
\frac{dR(t)}{dt} & = \gamma I(t)
\end{split}

\begin{split}
\frac{dD(t)}{dt} & = \mu I(t)
\end{split} \\

The idea is to linearize the derivative values from the measured dataset to get Beta, following the equation:

\begin{split}
\frac{\Delta S(t)}{\Delta t} & = \frac{-\beta I(t) S(t)}{N}
\end{split} 

In other words, we finally get the $\beta$ expression: 

\begin{split}
{\beta_i} & = -\frac{N*\Delta S/\Delta t}{ S_i * I_i}
\end{split}

Moreover, in order to implement this linearization to our study, the **population N** is not the real State population, since it leads to illegible results. From experience, we know that the best amount of **Susceptible population** is the sum of all 4 compartments at the end of the survey. \\
Furthermore, the idea is to complete the computation with a given **step $\Delta t$**, considering that calculating each $\beta_i$ for each day is not relevant, and leads to noisy results. \\
Hence, the **difference $\Delta S$** is, for a given time $t$, the value: $S[t+\Delta t] - S[t]$ \\
Finally, and for better precision, the quantities **$S_i$ and $I_i$** are calculated doing the mean of the values from $t$ to $t+\Delta t$. In other words: $S_i = \frac{\sum_{k=0}^{k=\Delta t} S[i*\Delta t + k]}{\Delta t}$ and $I_i = \frac{\sum_{k=0}^{k=\Delta t} I[i*\Delta t + k]}{\Delta t}$.


## **Data**

### DataFrames

>Creating a function returning the specific DataFrame for a Brazilian State.

In [ ]:
def state_df(State_name):

    # Loading data - wcota
  data_path = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
  df = pd.read_csv(data_path, delimiter=",") 

    # Dataframe for the specific
  df_state = df[df.state == State_name].reset_index()

    # Creating new recovered column
  df_state["newRecovered"] = df_state["recovered"].diff()
  df_state.newRecovered.fillna(0, inplace=True)
  df_state.recovered.fillna(0, inplace=True)

    # Creating active cases column (Infected)
  active_infected = [df_state["totalCases"].iloc[0]]
  for nc, nr in zip(df_state["newCases"].iloc[1:], 
                    df_state["newRecovered"].iloc[1:]):
      active_infected.append(active_infected[-1] + nc - nr)
  df_state["activeCases"] = active_infected

  df_state = df_state[['date','activeCases','recovered','deaths']]

  return df_state

>Example

In [ ]:
state_df('MA').head()

,date,activeCases,recovered,deaths
0,2020-03-20,1.0,0.0,0
1,2020-03-21,2.0,0.0,0
2,2020-03-22,2.0,0.0,0
3,2020-03-23,8.0,0.0,0
4,2020-03-24,8.0,0.0,0


### Formatting data to process

In [ ]:
#  Function formatting the data to process

def data_formatting(df_state):

  Infected = np.array(df_state['activeCases'])
  Recovered = np.array(df_state['recovered'])
  Deceased = np.array(df_state['deaths'])
  N = Infected[-1]+Recovered[-1]+Deceased[-1]
  Susceptible = np.array(N - Infected - Recovered)
  Date = np.array(df_state['date'])

  return Date, [Susceptible,Infected,Recovered,Deceased], N

In [ ]:
data = data_formatting(state_df('MA'))
graphics_plot(data[0], data[1], ['Susceptible','Infected','Recovered','Deceased'], 'Covid-19 Data Maranhao')

## **Beta Calculation**

In [ ]:
def beta_linearization(Date, SIRD_data, N, Step):
  Beta = []
  S,I,R,D = SIRD_data
  index_vector = range(0, len(Date)-Step, Step)
  for i in index_vector:
    deltaS = S[i+Step] - S[i]
    deltaT = Step
    Si, Ii = 0, 0
    for k in range(Step):
      Si += S[i+k]
      Ii += I[i+k]
    Si = Si/Step  #Caculating the mean to get S_i and I_i
    Ii = Ii/Step
    Beta += [abs(deltaS*N / (deltaT * Si * Ii))]
  time_scale = Date[index_vector]
  return time_scale, np.array(Beta)

In [ ]:
beta_linearization(data[0], data[1], data[2], 7)

(array(['2020-03-20', '2020-03-27', '2020-04-03', '2020-04-10',
        '2020-04-17', '2020-04-24', '2020-05-01', '2020-05-08',
        '2020-05-15', '2020-05-22', '2020-05-29', '2020-06-05',
        '2020-06-12', '2020-06-19', '2020-06-26', '2020-07-03',
        '2020-07-10', '2020-07-17', '2020-07-24', '2020-07-31',
        '2020-08-07', '2020-08-14', '2020-08-21', '2020-08-28',
        '2020-09-04', '2020-09-11', '2020-09-18', '2020-09-25',
        '2020-10-02', '2020-10-09', '2020-10-16', '2020-10-23',
        '2020-10-30', '2020-11-06', '2020-11-13', '2020-11-20',
        '2020-11-27', '2020-12-04', '2020-12-11', '2020-12-18',
        '2020-12-25', '2021-01-01', '2021-01-08', '2021-01-15',
        '2021-01-22', '2021-01-29', '2021-02-05', '2021-02-12',
        '2021-02-19', '2021-02-26', '2021-03-05', '2021-03-12',
        '2021-03-19', '2021-03-26', '2021-04-02', '2021-04-09',
        '2021-04-16', '2021-04-23', '2021-04-30', '2021-05-07',
        '2021-05-14', '2021-05-21', '202

In [ ]:
def beta_df(State_Name, Step):
  State_Data = data_formatting(state_df(State_Name))
  return beta_linearization(State_Data[0], State_Data[1], State_Data[2], Step)

In [ ]:
test = beta_df('MA', 7)
graphics_plot(test[0], [test[1]], ['Transmission Rate Beta'], 'Beta Linearization Curve - Maranhao State')

# **2nd Approach to get Beta: Gradient Descent**

A Second method to solve linear systems is the gradient descent. \\
This method is used to find a local minimum. This is really intuitive: this is an iterative algorithm which from a starting point is going down (in the opposite direction of the gradient), converging to the minimum.

# **3rd Approach to get Beta: Iterative method**

## Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy import optimize
import plotly.graph_objects as go
import yaml
from datetime import datetime
from sklearn import metrics as mt

## SIRD model

In [ ]:
# The SIRD model differential equations.
def SIRD(y, t, N, Beta, Gamma, Mu):
    S, I, R, D = y
    dSdt = -(Beta * I * S)/N
    dIdt = (Beta * I * S)/N  - Gamma * I - Mu * I
    dRdt = Gamma * I
    dDdt = Mu * I
    return dSdt, dIdt, dRdt, dDdt

In [ ]:
#Saving the simulation results

def SIRDsim(y0, t, N, theta):
  
  #Transmission rate
  Beta = theta[0]
  #Recovery rate per day
  Gamma = theta[1]
  #Mortality rate
  Mu = theta[2]

  # Integrate the SIRD equations over the time grid t.
  result = odeint(SIRD, y0, t, args=(N, Beta, Gamma, Mu))
  S, I, R, D = result.T
  return S, I, R, D

In [ ]:
#Least Squares Method

def QuadraticError(theta0, Sd, Id, Rd, Dd, y0, t, N):
    """ function to pass to optimize.leastsq
        The routine will square and sum the values returned by 
        this function""" 
    [S,I,R,D] = SIRDsim(y0, t, N, theta0)
    errorS = S - Sd
    errorI = I - Id
    errorR = R - Rd
    errorD = D - Dd
    EQ = np.concatenate([errorI,errorR,errorD])
    return EQ

## Parameters' Estimation Automation

> Brazil is composed of numerous States, and even if there is a National Healthcare System, known as the SUS, each State is governed independently and taken measures are not the same through the wide Brazilian territory. \\
Doing so, this study needs to consider each State independently, to correlate if the real epidemic's evolution is related to the taken measures or not. \\
To do so, automatizing the computational process is the best way to be efficient. 

In [ ]:
def Iterative_SIRD_beta(State_Name, Step=1):
  """This function is fitting the SIRD model to a Dataset
     and returns the best parameters through time.
     Args:
           - State_Name: Abbreviated name of a State  | string
           - Step: calculation step                   | int (days)
  """

  # Reading data - wcota
  data_path = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
  df = pd.read_csv(data_path, delimiter=",")

  # Dataframe for the specific State
  df_state = df[df.state == State_Name].reset_index()

  # Creating new recovered column
  df_state["newRecovered"] = df_state["recovered"].diff()
  df_state.newRecovered.fillna(0, inplace=True)
  df_state.recovered.fillna(0, inplace=True)

  # Creating active cases column (Infected)
  active_infected = [df_state["totalCases"].iloc[0]]
  for nc, nr in zip(df_state["newCases"].iloc[1:], 
                    df_state["newRecovered"].iloc[1:]):
      active_infected.append(active_infected[-1] + nc - nr)
  df_state["activeCases"] = active_infected

  #Susceptible population
  Tf = len(df_state.index)-1
  N = df_state["recovered"][Tf] + df_state["activeCases"][Tf] + df_state["deaths"][Tf]

  # Initial Conditions
  S0, I0, R0, D0 = N-1, 1, 0, 0
  y0 = S0, I0, R0, D0
  theta0 = [0.44, 0.15, 0.00292]

  # Vectors Initialization
  Beta_table =[]

  # Initialization of an index vector to personalize the computation
  Index_vector = range(Step,Tf,Step)
  # It starts from 10 to have enough samples to define parameters, stops at Tf, 
  # and use Step to change the frequency of the computation (since it takes a long time)

  # Time Loop to find parameters through time
  for Tresearch in Index_vector:
  
    # Considering the research duration for the parameters
    df_state_init = df_state.iloc[:Tresearch]
  
    # Data
    Id = df_state_init["activeCases"]
    Rd = df_state_init["recovered"]
    Dd = df_state_init["deaths"]
    Sd = N - Rd - Id - Dd

    # Time vector
    t = np.linspace(0, len(df_state_init.index.values), len(df_state_init.index.values))

    # Model use to find optimal parameters
    (best_theta, kvg) = optimize.leastsq(QuadraticError, theta0, args=(Sd,Id,Rd,Dd,y0,t,N))
    Beta_table.append(best_theta[0])

  timeline = df_state['date'].iloc[:Tf]
  data = {'Date': timeline}
  Parameters_df = pd.DataFrame(data)
  Parameters_df['Beta'] = pd.DataFrame(Beta_table, Index_vector)

  #Removing NaN rows due to uncalculated days between steps
  Parameters_df.dropna(subset=['Beta'], inplace=True)

  return Parameters_df

In [ ]:
SIRD_beta = Iterative_SIRD_beta('MA', 7)
SIRD_beta

,Date,Beta
7,2020-03-27,0.350794
14,2020-04-03,0.362978
21,2020-04-10,0.335334
28,2020-04-17,0.291721
35,2020-04-24,0.256922
...,...,...
483,2021-07-16,0.252373
490,2021-07-23,0.248666
497,2021-07-30,0.245045
504,2021-08-06,0.241580


>Since the observation that the computation is taking time (almost 40s if all samples are considered), the implementation of a step has been done. \\
The model can now offer the possibility to treat the data faster. \\
However, as illustrated above, gaining time is loosing accuracy, since the step is bigger. \\
Yet, this lost of accuracy would not be visually perceptible by plotting the results.

In [ ]:
graphics_plot(SIRD_beta['Date'], [SIRD_beta['Beta']], ['Iterative Beta'], 'Beta Iterative SIRD Curve - Maranhao State')

# **4th Approach to get Beta: the Observer**

# **4 Methods Synthesis**

The 4 methods being implemented, the purpose is now to compare all 4 methods to gain analyze experience and maybe develop a stronger accurate process getting the best of each Beta calculation.

In [ ]:
Linearization = beta_df('MA', 7)
Iterative_beta = Iterative_SIRD_beta('MA', 7)
graphics_plot(Linearization[0], [Linearization[1], Iterative_beta['Beta']], ['Linearized Beta Value','Iterative Beta Value'], 'Beta Estimation Curves - Maranhao State')

# **Model's verification**

Through this Notebook, we created several methods to determine and calculate the transmission rate $\beta$. \\
However, the only verification procedure we have is to compare the different curves each other. \\
To this extent, establishing a verification procedure is relevant for our study. The idea is to extract the calculated $\beta$ variatons and compute a simulation again, to compare the simulation with real data using the R² score.

## ***Modifications of the model to prepare simulation***

As we saw in the beginning of our project, we know that to simulate with the SIRD model, we need to provide the whole bunch of parameters. \\
Considering this, we need to complete the Beta study to compute all 3 parameters.

### ***1st Approach: Linearization to get every parameters***

A first and simple idea is to calculate $\beta$ with the given set of equations, replacing derivative values by linearized ones. 

#### **Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

#### **Utilitary Functions**

In [ ]:
def graphics_plot(t, y, y_title, title):
  # Plotting the results

  fig = go.Figure()

  for i in range(len(y)):
    if i<4:
      fig.add_trace(go.Scatter(
      name=y_title[i],
      x=t,
      y=y[i],
      mode='lines',
      marker_symbol='circle',
      line=dict(width=3, dash="dash")
      ))
    else:
      fig.add_trace(go.Scatter(
      name=y_title[i],
      x=t,
      y=y[i],
      mode='lines',
      marker_symbol='circle',
      line=dict(width=3, dash="solid")
      ))
  
  fig.update_layout(
      template='xgridoff',
      xaxis=dict(showgrid=False),
      xaxis_title='Date',
      legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.01,
        xanchor='right',
        x=0.95
        ),
      title_text=title)

  fig.show()

#### **Theory**

Regarding the equational system: 

$$ \begin{split}
   \frac{dS(t)}{dt} & = \frac{-\beta I(t) S(t)}{N}
   \end{split} $$

\begin{split}
\frac{dI(t)}{dt} & = \frac{\beta I(t) S(t)}{N} -\gamma I(t) -\mu I(t)
\end{split}

\begin{split}
\frac{dR(t)}{dt} & = \gamma I(t)
\end{split}

\begin{split}
\frac{dD(t)}{dt} & = \mu I(t)
\end{split} \\

The idea is to linearize the derivative values from the measured dataset to get Gamma and Mu, following the equations:

\begin{split}
\frac{\Delta R(t)}{\Delta t} & = {\gamma I(t)} \\
\frac{\Delta D(t)}{\Delta t} & = {\mu I(t)}
\end{split} 

In other words, we finally get the expressions: 
\begin{split}
{\beta_i} & = -\frac{N*\Delta S/\Delta t}{ S_i * I_i}
\end{split}
\begin{split}
{\gamma_i} & = \frac{\Delta R}{\Delta t * I_i} \\
{\mu_i} & = \frac{\Delta D}{\Delta t * I_i}
\end{split}

Moreover, the idea is to complete the computation with a given **step $\Delta t$**, considering that calculating each $\gamma_i$ and $\mu_i$ for each day is not relevant, and leads to noisy results. \\
Hence, the **differences $\Delta R$ and $\Delta D$** is, for a given time $t$, the respective values: **$R[t+\Delta t] - R[t] $ and $D[t+\Delta t] - D[t] $** \\
Finally, and for better precision, the quantity **$I_i$** is calculated doing the mean of the values from $t$ to $t+\Delta t$. In other words: $I_i = \frac{\sum_{k=0}^{k=\Delta t} I[i*\Delta t + k]}{\Delta t}$.


#### **Data**

##### DataFrames

>Creating a function returning the specific DataFrame for a Brazilian State.

In [ ]:
def state_df(State_name):

    # Loading data - wcota
  data_path = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
  df = pd.read_csv(data_path, delimiter=",") 

    # Dataframe for the specific
  df_state = df[df.state == State_name].reset_index()

    # Creating new recovered column
  df_state["newRecovered"] = df_state["recovered"].diff()
  df_state.newRecovered.fillna(0, inplace=True)
  df_state.recovered.fillna(0, inplace=True)

    # Creating active cases column (Infected)
  active_infected = [df_state["totalCases"].iloc[0]]
  for nc, nr in zip(df_state["newCases"].iloc[1:], 
                    df_state["newRecovered"].iloc[1:]):
      active_infected.append(active_infected[-1] + nc - nr)
  df_state["activeCases"] = active_infected

  df_state = df_state[['date','activeCases','recovered','deaths']]

  return df_state

>Example

In [ ]:
state_df('MA').head()

,date,activeCases,recovered,deaths
0,2020-03-20,1.0,0.0,0
1,2020-03-21,2.0,0.0,0
2,2020-03-22,2.0,0.0,0
3,2020-03-23,8.0,0.0,0
4,2020-03-24,8.0,0.0,0


##### Formatting data to process

In [ ]:
#  Function formatting the data to process

def data_formatting(df_state):

  Infected = np.array(df_state['activeCases'])
  Recovered = np.array(df_state['recovered'])
  Deceased = np.array(df_state['deaths'])
  N = Infected[-1]+Recovered[-1]+Deceased[-1]
  Susceptible = np.array(N - Infected - Recovered)
  Date = np.array(df_state['date'])

  return Date, [Susceptible,Infected,Recovered,Deceased], N

In [ ]:
data = data_formatting(state_df('MA'))
graphics_plot(data[0], data[1], ['Susceptible','Infected','Recovered','Deceased'], 'Covid-19 Data Maranhao')

#### **Parameters' Calculation**

In [ ]:
def parameters_linearization(Date, SIRD_data, N, Step):
  Beta = []
  Gamma = []
  Mu = []
  S,I,R,D = SIRD_data
  index_vector = range(0, len(Date)-Step, Step)
  for i in index_vector:
    deltaS = S[i+Step] - S[i]
    deltaR = R[i+Step] - R[i]
    deltaD = D[i+Step] - D[i]
    deltaT = Step
    Si, Ii = 0, 0
    for k in range(Step):
      Si += S[i+k]
      Ii += I[i+k]
    Si = Si/Step  #Caculating the mean to get S_i and I_i
    Ii = Ii/Step
    Beta += [abs(deltaS*N / (deltaT * Si * Ii))]
    Gamma += [abs(deltaR / (deltaT * Ii))]
    Mu += [abs(deltaD / (deltaT * Ii))]
  time_scale = Date[index_vector]
  return time_scale, np.array(Beta), np.array(Gamma), np.array(Mu)

In [ ]:
def parameters_df(State_Name, Step):
  State_Data = data_formatting(state_df(State_Name))
  return parameters_linearization(State_Data[0], State_Data[1], State_Data[2], Step)

In [ ]:
test = parameters_df('MA', 7)
graphics_plot(test[0], [test[1],test[2],test[3]], ['Transmission Rate Beta','Recovery Rate Gamma','Mortality Rate Mu'], 'Parameters Linearization Curves - Maranhao State')

### **3rd Approach to get Beta: Iterative method**

#### Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy import optimize
import plotly.graph_objects as go
import yaml
from datetime import datetime
from sklearn import metrics as mt

#### SIRD model

In [ ]:
# The SIRD model differential equations.
def SIRD(y, t, N, Beta, Gamma, Mu):
    S, I, R, D = y
    dSdt = -(Beta * I * S)/N
    dIdt = (Beta * I * S)/N  - Gamma * I - Mu * I
    dRdt = Gamma * I
    dDdt = Mu * I
    return dSdt, dIdt, dRdt, dDdt

In [ ]:
#Saving the simulation results

def SIRDsim(y0, t, N, theta):
  
  #Transmission rate
  Beta = theta[0]
  #Recovery rate per day
  Gamma = theta[1]
  #Mortality rate
  Mu = theta[2]

  # Integrate the SIRD equations over the time grid t.
  result = odeint(SIRD, y0, t, args=(N, Beta, Gamma, Mu))
  S, I, R, D = result.T
  return S, I, R, D

In [ ]:
#Least Squares Method

def QuadraticError(theta0, Sd, Id, Rd, Dd, y0, t, N):
    """ function to pass to optimize.leastsq
        The routine will square and sum the values returned by 
        this function""" 
    [S,I,R,D] = SIRDsim(y0, t, N, theta0)
    errorS = S - Sd
    errorI = I - Id
    errorR = R - Rd
    errorD = D - Dd
    EQ = np.concatenate([errorI,errorR,errorD])
    return EQ

#### Parameters' Estimation Automation

> Brazil is composed of numerous States, and even if there is a National Healthcare System, known as the SUS, each State is governed independently and taken measures are not the same through the wide Brazilian territory. \\
Doing so, this study needs to consider each State independently, to correlate if the real epidemic's evolution is related to the taken measures or not. \\
To do so, automatizing the computational process is the best way to be efficient. 

In [ ]:
def Iterative_SIRD_beta(State_Name, Step=1):
  """This function is fitting the SIRD model to a Dataset
     and returns the best parameters through time.
     Args:
           - State_Name: Abbreviated name of a State  | string
           - Step: calculation step                   | int (days)
  """

  # Reading data - wcota
  data_path = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
  df = pd.read_csv(data_path, delimiter=",")

  # Dataframe for the specific State
  df_state = df[df.state == State_Name].reset_index()

  # Creating new recovered column
  df_state["newRecovered"] = df_state["recovered"].diff()
  df_state.newRecovered.fillna(0, inplace=True)
  df_state.recovered.fillna(0, inplace=True)

  # Creating active cases column (Infected)
  active_infected = [df_state["totalCases"].iloc[0]]
  for nc, nr in zip(df_state["newCases"].iloc[1:], 
                    df_state["newRecovered"].iloc[1:]):
      active_infected.append(active_infected[-1] + nc - nr)
  df_state["activeCases"] = active_infected

  #Susceptible population
  Tf = len(df_state.index)-1
  N = df_state["recovered"][Tf] + df_state["activeCases"][Tf] + df_state["deaths"][Tf]

  # Initial Conditions
  S0, I0, R0, D0 = N-1, 1, 0, 0
  y0 = S0, I0, R0, D0
  theta0 = [0.44, 0.15, 0.00292]

  # Vectors Initialization
  Beta_table =[]
  Gamma_table = []
  Mu_table = []

  # Initialization of an index vector to personalize the computation
  Index_vector = range(Step,Tf,Step)
  # It starts from 10 to have enough samples to define parameters, stops at Tf, 
  # and use Step to change the frequency of the computation (since it takes a long time)

  # Time Loop to find parameters through time
  for Tresearch in Index_vector:
  
    # Considering the research duration for the parameters
    df_state_init = df_state.iloc[:Tresearch]
  
    # Data
    Id = df_state_init["activeCases"]
    Rd = df_state_init["recovered"]
    Dd = df_state_init["deaths"]
    Sd = N - Rd - Id - Dd

    # Time vector
    t = np.linspace(0, len(df_state_init.index.values), len(df_state_init.index.values))

    # Model use to find optimal parameters
    (best_theta, kvg) = optimize.leastsq(QuadraticError, theta0, args=(Sd,Id,Rd,Dd,y0,t,N))
    Beta_table.append(best_theta[0])
    Gamma_table.append(best_theta[1])
    Mu_table.append(best_theta[2])

  timeline = df_state['date'].iloc[:Tf]
  data = {'Date': timeline}
  Parameters_df = pd.DataFrame(data)
  Parameters_df['Beta'] = pd.DataFrame(Beta_table, Index_vector)
  Parameters_df['Gamma'] = pd.DataFrame(Gamma_table, Index_vector)
  Parameters_df['Mu'] = pd.DataFrame(Mu_table, Index_vector)

  #Removing NaN rows due to uncalculated days between steps
  Parameters_df.dropna(subset=['Beta'], inplace=True)

  return Parameters_df

In [ ]:
SIRD_beta = Iterative_SIRD_beta('MA', 7)
SIRD_beta

,Date,Beta,Gamma,Mu
7,2020-03-27,0.350794,-0.000006,0.000041
14,2020-04-03,0.362978,0.038085,0.005202
21,2020-04-10,0.335334,0.048790,0.013437
28,2020-04-17,0.291721,0.036138,0.014949
35,2020-04-24,0.256922,0.025834,0.011010
...,...,...,...,...
483,2021-07-16,0.252373,0.160709,0.004244
490,2021-07-23,0.248666,0.157397,0.004173
497,2021-07-30,0.245045,0.154171,0.004103
504,2021-08-06,0.241580,0.151088,0.004037


>Since the observation that the computation is taking time (almost 40s if all samples are considered), the implementation of a step has been done. \\
The model can now offer the possibility to treat the data faster. \\
However, as illustrated above, gaining time is loosing accuracy, since the step is bigger. \\
Yet, this lost of accuracy would not be visually perceptible by plotting the results.

In [ ]:
graphics_plot(SIRD_beta['Date'], [SIRD_beta['Beta'],SIRD_beta['Gamma'],SIRD_beta['Mu']], ['Iterative Beta','Iterative Gamma','Iterative Mu'], 'Parameters Iterative SIRD Curves - Maranhao State')

## ***New simulations from the obtained parameters to compare with real data***

Now that we complexified the model to get the 3 parameters, we can simulate the SIRD curves again, to verify if the model is strong enough.

### ***1st method: Iterative SIRD simulation by segment***

However, we only know how to simulate and plot simulations from a single bunch of parameters, not varying as the ones we now have. To solve this problem and try to observe relevant results, a possibility could be to simulate the SIRD model using segments of time. In other words, we need to each time simulate during the 'step' window.

#### *SIRD Model*

In [ ]:
# The SIRD model differential equations.
def SIRD(y, t, N, Beta, Gamma, Mu):
    S, I, R, D = y
    dSdt = -(Beta * I * S)/N
    dIdt = (Beta * I * S)/N  - Gamma * I - Mu * I
    dRdt = Gamma * I
    dDdt = Mu * I
    return dSdt, dIdt, dRdt, dDdt

In [ ]:
#Saving the simulation results

def SIRDsim(y0, t, N, theta):
  
  #Transmission rate
  Beta = theta[0]
  #Recovery rate per day
  Gamma = theta[1]
  #Mortality rate
  Mu = theta[2]

  # Integrate the SIRD equations over the time grid t.
  result = odeint(SIRD, y0, t, args=(N, Beta, Gamma, Mu))
  S, I, R, D = result.T
  return S, I, R, D

#### *Simulation & Verification*

In [ ]:
def SIRDverification(State_name, parameters_df, Step):

  # Getting the real data
  df_state = state_df(State_name) 
  df_state_array = data_formatting(df_state)

  # Initialization
  df_simulation = df_state[['date']]
  Is = [1]
  Rs = [0]
  Ds = [0]
  Ss = [df_state_array[1][0][0]]

  # Simulation
  for Increment in parameters_df.index.values:

    df_state_sim = df_state.iloc[Increment-Step:Increment] # Days of simulation
    df_state_sim_array = data_formatting(df_state_sim) # Getting Date, [S,I,R,D], N
    tsim = np.linspace(0, len(df_state_sim.index.values), len(df_state_sim.index.values))
    
    S0 = Ss[-1]
    I0 = Is[-1]
    R0 = Rs[-1]
    D0 = Ds[-1]
    y0 = S0, I0, R0, D0

    theta = parameters_df['Beta'][Increment],parameters_df['Gamma'][Increment],parameters_df['Mu'][Increment]

    [Ss_Increment,Is_Increment,Rs_Increment,Ds_Increment] = SIRDsim(y0, tsim, df_state_sim_array[2], theta)

    Ss += Ss_Increment.tolist()
    Is += Is_Increment.tolist()
    Rs += Rs_Increment.tolist()
    Ds += Ds_Increment.tolist()

  df_simulation = df_simulation.iloc[:len(Ss)]

  df_simulation['Ss'] = Ss
  df_simulation['Is'] = Is
  df_simulation['Rs'] = Rs
  df_simulation['Ds'] = Ds

  return df_state_array[1], df_simulation

##### *Verification with 1st: Linearization*

In [ ]:
new = pd.DataFrame(np.transpose(test), index=np.arange(7,7*(len(test[0])+1),7) , columns=['Date','Beta','Gamma','Mu'])

In [ ]:
Real_data, Simulation = SIRDverification('MA', new, 7)

In [ ]:
graphics_plot(Simulation['date'], [Real_data[0],Real_data[1],Real_data[2],Real_data[3],Simulation['Ss'],Simulation['Is'],Simulation['Rs'],Simulation['Ds']], ['Susceptible - data','Infected - data','Recovered - data','Deceased - data','Susceptible - Sim','Infected - Sim','Recovered - Sim','Deceased - Sim'], 'Comparison between Real Data and Simulation from obtained parameters - Maranhao State')

In [ ]:
from sklearn import metrics

# Shaping real data into DataFrame
Real_data_df = pd.DataFrame(np.transpose(Real_data)[:len(Simulation['Ss'])], columns=['Ss','Is','Rs','Ds'])

metrics.r2_score(Real_data_df, Simulation[['Ss','Is','Rs','Ds']])

-103.14036118544007

##### *Verification with 3rd: Iterative method*

In [ ]:
Real_data, Simulation = SIRDverification('MA', SIRD_beta, 7)

In [ ]:
graphics_plot(Simulation['date'], [Real_data[0],Real_data[1],Real_data[2],Real_data[3],Simulation['Ss'],Simulation['Is'],Simulation['Rs'],Simulation['Ds']], ['Susceptible - data','Infected - data','Recovered - data','Deceased - data','Susceptible - Sim','Infected - Sim','Recovered - Sim','Deceased - Sim'], 'Comparison between Real Data and Simulation from obtained parameters - Maranhao State')

In [ ]:
from sklearn import metrics

# Shaping real data into DataFrame
Real_data_df = pd.DataFrame(np.transpose(Real_data)[:len(Simulation['Ss'])], columns=['Ss','Is','Rs','Ds'])

metrics.r2_score(Real_data_df, Simulation[['Ss','Is','Rs','Ds']])

-143.58103521550856

### ***2nd method: Linearization and equational system's solving***

When analyzing the results of the simulation, we can see that using the iterative SIRD simulation by segment is not relevant. Indeed, the curves obtained are not smooth, and do not fit to real data at all. \\
Looking for solutions to solve this issue and try to get well fitted simulations, the idea to linearize data again from the real dataset, and calculate the S, I, R and D values came.

#### **Theory**

Considering the initial equational system:

$$ \begin{split}
   \frac{dS(t)}{dt} & = \frac{-\beta I(t) S(t)}{N}
   \end{split} $$

\begin{split}
\frac{dI(t)}{dt} & = \frac{\beta I(t) S(t)}{N} -\gamma I(t) -\mu I(t)
\end{split}

\begin{split}
\frac{dR(t)}{dt} & = \gamma I(t)
\end{split}

\begin{split}
\frac{dD(t)}{dt} & = \mu I(t)
\end{split} \\

Linearizing the derivative values, we have: \\

\begin{split}
I_i & = \frac{\Delta R(t)}{\Delta t*\gamma_i} \
or \ I_i = \frac{\Delta D(t)}{\Delta t*\mu_i} \\
and \ S_i & = \frac{N}{\beta_i}[\frac{\Delta I(t)}{I_i*\Delta t} + (\gamma_i + \mu_i)]
\end{split}

#### **Code**

In [ ]:
def linear_simulation(State_Name, parameters_df, Step):
  
  # Initialization
  SIRD_df = state_df(State_Name)
  Date, [S,I,R,D], N = data_formatting(SIRD_df) #Getting Date,[SIRD],N to compute deltaR, deltaD, and use N
  I_s_gamma = [1]
  I_s_mu = [1]
  S_s = [N]
  simulation_df = parameters_df[['Date']]
  deltaT = Step

  for Index in parameters_df.index.values:

    beta_i = parameters_df['Beta'][Index]
    gamma_i = parameters_df['Gamma'][Index]
    mu_i = parameters_df['Mu'][Index]

    deltaR = R[Index] - R[Index-Step]
    deltaD = D[Index] - D[Index-Step]

    I_s_gamma += [deltaR/(deltaT*gamma_i)]
    I_s_mu += [deltaD/(deltaT*mu_i)]

    I_i = I_s_gamma[-1]
    deltaI = I[Index] - I[Index-Step]

    S_s += [N*(deltaI/(I_i*deltaT) + gamma_i + mu_i)/beta_i]

  simulation_df['Ss'] = S_s[1:]
  simulation_df['I_gamma_s'] = I_s_gamma[1:]
  simulation_df['I_mu_s'] = I_s_mu[1:]

  return simulation_df, [S,I]

#### **Computation and Visualization**

##### *Verification with 1st: Linearization*

In [ ]:
new = pd.DataFrame(np.transpose(test), index=np.arange(7,7*(len(test[0])+1),7) , columns=['Date','Beta','Gamma','Mu'])

In [ ]:
linear_simulation, data_df = linear_simulation('MA', new, 7)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
graphics_plot(linear_simulation['Date'], [linear_simulation['Ss'],linear_simulation['I_gamma_s'],linear_simulation['I_mu_s'],data_df[0].tolist()[7:511:7],data_df[1].tolist()[7:511:7]], ['Susceptible - sim','Infected gamma - sim','Infected mu - sim','Susceptible - data','Infected - data'], 'Comparison between data and simulation from obtained parameters - Maranhao State')

In [ ]:
linear_simulation

,Date,Ss,I_gamma_s,I_mu_s
7,2020-03-20,NaN,NaN,NaN
14,2020-03-27,359142.647841,40.000000,40.000000
21,2020-04-03,382479.105614,167.857143,167.857143
28,2020-04-10,366219.964080,479.571429,479.571429
35,2020-04-17,372168.240778,1335.428571,1335.428571
...,...,...,...,...
483,2021-07-09,29110.739833,44304.285714,44304.285714
490,2021-07-16,NaN,NaN,45179.428571
497,2021-07-23,21495.379203,45396.428571,45396.428571
504,2021-07-30,17801.502325,48072.285714,48072.285714


In [ ]:
from sklearn import metrics

Sd = data_df[0][7:511:7]
Id = data_df[1][7:511:7]

# Ss and Is have Nan rows, replacing them with 
linear_simulation.at[490,'Ss'] = data_df[0][490]
linear_simulation.at[490,'I_gamma_s']=data_df[1][490]

Ss = np.array(linear_simulation['Ss'][1:])
Is = np.array(linear_simulation['I_gamma_s'][1:])

metrics.r2_score([Sd,Id], [Ss,Is])

0.9498335505191496

##### *Verification with 3rd: Iterative SIRD*

In [ ]:
linear_simulation2, data_df2 = linear_simulation('MA', SIRD_beta, 7)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning:

divide by zero encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
graphics_plot(linear_simulation2['Date'], [linear_simulation2['Ss'],linear_simulation2['I_gamma_s'],linear_simulation2['I_mu_s'],data_df2[0].tolist()[7:511:7],data_df2[1].tolist()[7:511:7]], ['Susceptible - sim','Infected gamma - sim','Infected mu - sim','Susceptible - data','Infected - data'], 'Comparison between data and simulation from obtained parameters - Maranhao State')

In [ ]:
from sklearn import metrics

Sd = data_df2[0][7:511:7]
Id = data_df2[1][7:511:7]

# Ss and Is have Nan rows, replacing them with 
linear_simulation2.at[490,'Ss'] = data_df2[0][490]
linear_simulation2.at[490,'I_gamma_s']=data_df2[1][490]

Ss = np.array(linear_simulation2['Ss'][1:])
Is = np.array(linear_simulation2['I_gamma_s'][1:])

metrics.r2_score([Sd,Id], [Ss,Is])

-1065.1794126298332

### ***3rd method: Time dependent parameters ODE solving***

Now that we tried two methods in order to simulate the SIRD model with time dependent parameters, and since the results are not satisfying, we thought with the team that we could use a new way to simulate, with time dependent parameters.

#### **Theory**

If we consider the initial equational system, we can see that we could just modify the already implemented SIRD model, using a function to return the $\beta(t)$ parameter considering time t, then solve this model to simulate the curves.

$$ \begin{split}
   \frac{dS(t)}{dt} & = \frac{-\beta(t) I(t) S(t)}{N}
   \end{split} $$

\begin{split}
\frac{dI(t)}{dt} & = \frac{\beta(t) I(t) S(t)}{N} -\gamma (t) I(t) -\mu (t) I(t)
\end{split}

\begin{split}
\frac{dR(t)}{dt} & = \gamma (t) I(t)
\end{split}

\begin{split}
\frac{dD(t)}{dt} & = \mu (t) I(t)
\end{split} \\

#### *Time dependent SIRD Model*

In [ ]:
def Beta(Parameters_df, t):
  return Parameters_df['Beta'][t]

In [ ]:
def Gamma(Parameters_df, t):
  return Parameters_df['Gamma'][t]

In [ ]:
def Mu(Parameters_df, t):
  return Parameters_df['Mu'][t]

In [ ]:
# The SIRD model differential equations.
def SIRD(y, t, N, Parameters_df):
    S, I, R, D = y
    dSdt = -(Beta(Parameters_df, t) * I * S)/N
    dIdt = (Beta(Parameters_df, t) * I * S)/N  - Gamma(Parameters_df, t) * I - Mu(Parameters_df, t) * I
    dRdt = Gamma(Parameters_df, t) * I
    dDdt = Mu(Parameters_df, t) * I
    return dSdt, dIdt, dRdt, dDdt

In [ ]:
#Saving the simulation results

def SIRDsim(y0, t, N, theta):
  
  #Transmission rate
  Beta = theta[0]
  #Recovery rate per day
  Gamma = theta[1]
  #Mortality rate
  Mu = theta[2]

  # Integrate the SIRD equations over the time grid t.
  result = odeint(SIRD, y0, t, args=(N, Beta, Gamma, Mu))
  S, I, R, D = result.T
  return S, I, R, D